# User's Manual for Land-Use Trade-Offs II model

LUTO-2 (hereafter referred to as LUTO) is version two of the Land-Use Trade-Offs model. The model is an agricultural model comprised by 4 parts: 1) Land-use map, 2) Productivity maps, and 3) Resources (water, monetary, biodiversity) consumption maps and 4) an optimizer. LUTO predicts future spatial land-use distributions based on an economic cost-minimisation or profit-maximization logic under various constraints. 

The optimization process can be primarily divided into 3 steps:
1) Find the solutions that meet the demand<br>
    `Land-use map` * `Productivity map` >= `Demand`
2) Find the solutions that meet the environmental constraints<br>
    `Land-use map` * `Resource requirement map` <= `Constraint`
3) Maximize profit (or minimize cost) to get the optimal solution<br>
    `Best solution` = `Maximize(Revenue - Cost)` or `Minimize(Cost)`

LUTO is designed as a policy tool to answer 'what-if' questions under major climate change and socio-ecnomicy pathways, as well as Australia's national context. The `Demand` is sourced from external modelling considering Australia's imports, exports, diets, et cetera; the `productivity map` is driven by different climate change predictions; and the `resource requirement map` is acquired through paper review and stakeholder collaborations. Users can specify their assumptions by choosing parameters in the `settings.py` file. 

As an highly optimized model, LUTO rearranges all the input data from 2D-map format into a 1D-array representation to save memory and reduce running time, making it difficult for users to intuitively understand what happened within each process. This manual converts the 1D array back to its original 2D format and visualize the process of arranging land-use map to meet demand targets under water, GHG, and biodiversity constraints.

```{note}
LUTO-2 is capable of running land-use change for Australia at a ~1x1 square kilometer scale, but this chapter will use a setting that specifies `RESFACTOR=5` to reduce processing time. Essentially, this resfactor means we are only using the centroid cell out of each 5*5 square matrix to run the model.
```

Before entering the amazing LUTO world, please keep in mind that this manual tries to explain how LUTO works in the most intuitive way. That means sometimes we may lose technical depth or skip over certain complexities. If you are keen to understand how exactly this model works, we encourage you to look at the source code and delve into the tech/data manual provided under the `/docs`. 

In [ ]:
# REMOVE

'''
Here are the code to run LUTO first, so that we can have the variables and objects required for illustration in the notebook.
'''

# Reload all modules before excuting each cell, so that we can modify the imported custom functions on-the-fly without rerun the whole model.
%load_ext autoreload
%autoreload 2

# Imports
import os
import h5py           
import netCDF4  
from lxml import etree

import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotnine as p9
import geopandas as gpd

from rasterstats import zonal_stats
from itertools import product
from tqdm.auto import tqdm

# Set pandas display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)


# cd to the root folder of LUTO project
os.chdir('N:/LUF-Modelling/LUTO2_JZ/luto-2.0')
from luto.tools.Manual_jupyter_books.helpers import map_to_4band, mrj_to_xr, arr_to_xr, map_to_plot
from luto.economics.agricultural.quantity import get_quantity_matrices
from luto.economics.agricultural.water import get_wreq_matrices
from luto.tools.Manual_jupyter_books.helpers.parameters import legend_params


# # Run LUTO
# import luto.simulation as sim
# data = sim.load_data()
# sim.run(data=data, base=2010, target=2050)